In [ ]:
import requests
import time
from fair_research_login import NativeClient

client = NativeClient(client_id='7414f0b4-7d05-4bb6-bb00-076fa3f17cf5')
tokens = client.login(
    requested_scopes=['urn:globus:auth:scope:transfer.api.globus.org:all',
                      "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/all", 
                      'email', 'openid'],
    no_local_server=True,
    no_browser=True)

transfer_token = tokens['transfer.api.globus.org']['access_token']
funcx_token = tokens['funcx_service']['access_token']
headers = {'Authorization': f"Bearer {funcx_token}",'Transfer': transfer_token, 'FuncX': f"{funcx_token}"}
print(f"Headers: {headers}")

In [ ]:
url = "http://127.0.0.1:5000/upload_def_file"
definition_ids = []

for i in range(3):
    file_name = "Dockerfile"
    file_path = "Dockerfile"
    my_files = {"file": (file_name, open(file_path,"rb"))}
    response = requests.post(url, files=my_files, headers=headers)
    definition_id = response.text
    definition_ids.append(definition_id)
    print(definition_id)

print(definition_ids)

In [ ]:
url = "http://127.0.0.1:5000/build"
build_ids = []

for idx, definition_id in enumerate(definition_ids):
    my_data = {"definition_id": definition_id, "to_format": "docker", 
               "container_name": "my_container_{}".format(idx)}
    response = requests.post(url, json=my_data, headers=headers)
    build_id = response.text
    build_ids.append(build_id)
    print(build_id)


print(build_ids)

In [ ]:
url = "http://127.0.0.1:5000/build"
keep_printing = True
import time
import json
from IPython.display import clear_output
while keep_printing:
    clear_output(True)
    is_done = []
    for idx, build_id in enumerate(build_ids):
        response = requests.get(url, json={"build_id": build_id} , headers=headers)
        if json.loads(response.text)["build_status"] == "success":
            is_done.append(True)
        else:
            is_done.append(False)
        print(time.time())
        print(response.text)
    if all(is_done):
        keep_printing = False
    time.sleep(20)

## Stuff to work on for next time:
 - Try to see if restarting docker fixes the 500 Server Error: Internal Server Error ("a disk usage operation is already running") error
 - Add some sort of cleanup for when the build fails
 - Add retrying stuff
 - Scale big time